# Text Analytics


https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/quickstarts/client-libraries-rest-api?tabs=version-3-2&pivots=rest-api

In [1]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json
import traceback

# https://{endpoint}/text/analytics/v3.1/sentiment[?model-version][&showStats][&loggingOptOut][&opinionMining][&stringIndexType]
def get_sentiment_analysis_cs(text: str):
    assert text
    
    headers = {
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '', ## TODO
    }

    params = urllib.parse.urlencode({
        'model-version': 'latest',
        'showStats': 'true',
        'loggingOptOut': 'true',
        'opinionMining': 'true',
        'stringIndexType': 'TextElement_v8',
    })

    body = """{
        documents: [
            {
                id: "1",
                "language": "en",
                text: " """ + text + """ "
            }
        ]
    }"""

    try:
        conn = http.client.HTTPSConnection('eastus.api.cognitive.microsoft.com')  ## TODO
        conn.request("POST", "/text/analytics/v3.1/sentiment?%s" % params, body.encode('utf-8'), headers)
        response = conn.getresponse()
        data = response.read()
        conn.close()
        return(json.loads(data))
    except Exception:
        traceback.print_exc()

StatementMeta(new, 74, 1, Finished, Available)

# Bing News Search

https://docs.microsoft.com/en-us/azure/cognitive-services/bing-news-search/python

In [2]:
search_term = 'microsoft'
project_name = 'microsoft'

StatementMeta(new, 74, 2, Finished, Available)

In [3]:
import requests
import json

subscription_key = "" # TODO
search_url = "https://eastus.api.cognitive.microsoft.com/bing/v7.0/news/search" # TODO

headers_2 = {"Ocp-Apim-Subscription-Key" : subscription_key}
params_2  = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}

response = requests.get(search_url, headers=headers_2, params=params_2)

response.raise_for_status()

search_results = response.json()

descriptions = [article["description"] for article in search_results["value"]]

StatementMeta(new, 74, 3, Finished, Available)

In [4]:
import unicodedata

df = descriptions

# https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize

# df = [str(unicodedata.normalize('NFKD', text).encode('ascii', 'ignore')) for text in df] 
# df = [text.replace("&#39;", "'") for text in df]
df = [text.replace('<b>', ' ') for text in df]
df = [text.replace('</b>', ' ') for text in df]
# df = [text.replace("&quot;", ' ') for text in df]
# df = [text.replace("<b>", ' ') for text in df]
# df = [text.replace("</b>", ' ') for text in df]


for d in df:
    print(d)


StatementMeta(new, 74, 4, Finished, Available)

Microsoft Ireland has found that nearly half of Irish organisations don&#39;t have the right digital services to handle new working realities.
Some Microsoft counterfeit software users are seeing a promotional banner for 50 percent off. Clicking the link takes you to this page, which explains the risks
Rather than threatening Office pirates with legal action, Microsoft is trying a new tactic: offer them a substantial discount to go legal. As Ghacks reports, Microsoft has been spotted detecting when a pirate copy of Microsoft Office is being used and popping-up a notification just below the ribbon bar.
Microsoft has offered a 50% discount on its cloud-based Microsoft 356 Personal and Family subscriptions as it continues its efforts to turn software pirates into paying customers. The company is nudging users using unlicensed Microsoft Office on their computers to subscribe at a discounted rate for the first year.
US Justice Department has reached a settlement with  Microsoft  Corp resolv

In [5]:
des_df = df
sentiment = []
positive = []
neutral = []
negative = []

for single_news in des_df:
    print(str(single_news))
    r = get_sentiment_analysis_cs(str(single_news))
    # print("\n")
    # print(r)
    for i in r['documents']:
        sentiment.append(i['sentiment'])
        positive.append(i['confidenceScores']['positive'])
        neutral.append(i['confidenceScores']['neutral'])
        negative.append(i['confidenceScores']['negative'])

    # print(res['documents'])
    # sentiment.append(res['documents'])

StatementMeta(new, 74, 5, Finished, Available)

Microsoft Ireland has found that nearly half of Irish organisations don&#39;t have the right digital services to handle new working realities.
Some Microsoft counterfeit software users are seeing a promotional banner for 50 percent off. Clicking the link takes you to this page, which explains the risks
Rather than threatening Office pirates with legal action, Microsoft is trying a new tactic: offer them a substantial discount to go legal. As Ghacks reports, Microsoft has been spotted detecting when a pirate copy of Microsoft Office is being used and popping-up a notification just below the ribbon bar.
Microsoft has offered a 50% discount on its cloud-based Microsoft 356 Personal and Family subscriptions as it continues its efforts to turn software pirates into paying customers. The company is nudging users using unlicensed Microsoft Office on their computers to subscribe at a discounted rate for the first year.
US Justice Department has reached a settlement with  Microsoft  Corp resolv

In [6]:
results = sqlContext.createDataFrame(zip(des_df, sentiment, positive, neutral, negative), schema=['news', 'sentiment', 'positive', 'neutral', 'negative'])

ADLS_PATH = "abfss://container@storage_account.dfs.core.windows.net/fb-scraper/" + project_name + "/silver/"

results.write.format("csv").option("header","true").option("sep",",").mode("overwrite").save(ADLS_PATH + 'news_sentiment.csv')

# results.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv(ADLS_PATH + 'news_sentiment.csv')

# results.show()


StatementMeta(new, 74, 6, Finished, Available)